# Clean Strava & Weather Data
* Wrangle Strava JSON data into a pandas dataframe 
* Refine and create features from Strava data
* Clean data scraped from Weather Underground
* Join cleansed Strava & WUnderground data sets and write to file

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
data_fpath = '/Users/bkhurley/git/diablo_velo/data/'
rawdata_fpath = data_fpath + 'raw_data/'
cleandata_fpath = data_fpath + 'clean_data/'

## Load JSON data

In [3]:
# load effort data
segment_efforts = pickle.load(open(rawdata_fpath + "all_efforts_656860.p", "rb"))

# load segment data (data on the segment itself, not the efforts)
segment_data = pickle.load(open(rawdata_fpath + "segment_656860.p", "rb"))
segment_data = segment_data

## Restructure data from JSON to dataframe
Let's reformat the data from a JSON object to a pandas dataframe, which is more suitable for analysis.

As we go, keep only features that may be useful to the analysis.

In [4]:
# features to keep
features = ['id', 'activity', 'athlete', 'elapsed_time', 'moving_time', 'start_date',
            'start_date_local', 'distance', 'start_index', 'end_index', 'device_watts', 
            'average_watts', 'average_heartrate', 'max_heartrate', 'average_cadence']

I parse the JSON data by iterating through each effort, and within each effort iterating through each attribute of the effort. Each of the JSON object's (retained) dictionary keys become columns, each row listing the value of those column variables for a given effort.

In [14]:
# initiate empty list to hold rows of data
effort_data = []

# now, iterate through each effort
for effort in segment_efforts:    
    row_array = []
    # within each effort, iterate through each variable
    for i in range(len(features)):
        # only process features of interest
        attribute = features[i]
        if attribute not in effort.keys():
            row_array.append(None)
        else:
            # 'activity' and 'athlete' keys contain dicts. Keep only id from those dicts
            if attribute in ['activity', 'athlete']:
                if type(effort[attribute]) == dict:
                    effort[attribute] = effort[attribute]['id']
            # convert id numbers to strings for easier grouping later
            if attribute in ['id', 'activity', 'athlete']:
                effort[attribute] = str(effort[attribute])
            row_array.append(effort[attribute])
    
    # append row to data list 
    effort_data.append(row_array)

# convert data list to pandas dataframe
strava_data = pd.DataFrame(effort_data, columns=features)

Let's take a look at the new dataframe

In [15]:
strava_data.head()

,id,activity,athlete,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,device_watts,average_watts,average_heartrate,max_heartrate,average_cadence
0,9131432,111986,5287,4578,4310,2006-04-23T16:06:17Z,2006-04-23T16:06:17Z,15183.9,450,1052,False,NaN,149.5,164.0,NaN
1,9131428,125030,1885,5627,5160,2006-05-06T15:40:44Z,2006-05-06T15:40:44Z,16799.1,327,1095,False,212.9,175.1,186.0,66.1
2,1670259583,82155037,1885,5494,5255,2006-06-09T15:50:57Z,2006-06-09T08:50:57Z,16920.2,112,931,False,201.6,165.7,175.0,67.9
3,24297526436,991534814,3719,5320,5320,2006-07-04T16:59:01Z,2006-07-04T09:59:01Z,15467.1,149,350,False,182.8,NaN,NaN,NaN
4,1670314655,82158071,1885,5153,5033,2006-07-29T15:30:41Z,2006-07-29T08:30:41Z,16898.2,272,1079,False,226.6,172.7,180.0,73.9


## Clean the data
Now, I will clean up the data by removing unwanted columns, refining some features, and generating new features.

### Remove columns that we probably won't need
We will go with the local start date, and the start and stop indices don't appear to be very informative to our needs.

In [16]:
del strava_data['start_date']
del strava_data['start_index']
del strava_data['end_index']

### Clean up dates & times

In [17]:
# convert start date to pandas' datetime format
strava_data['start_date_local'] = pd.to_datetime(strava_data['start_date_local'])

### Generate some extra features from the data we have

#### Date & time features
I'll break out the timestamp into various date and time features I can easily index later.

In [18]:
strava_data['date'] = strava_data['start_date_local'].apply(lambda x: x.date())
strava_data['time'] = strava_data['start_date_local'].apply(lambda x: x.time())
strava_data['day'] = strava_data['start_date_local'].apply(lambda x: x.weekday_name)
strava_data['hour'] = strava_data['start_date_local'].apply(lambda x: x.hour)
strava_data['month'] = strava_data['start_date_local'].apply(lambda x: x.month)
strava_data['year'] = strava_data['start_date_local'].apply(lambda x: x.year)

#### Performance features
Next, I'll work on features related to athletes' cycling performance.

I'll start by transforming 'distance' from meters to miles, calculating miles per hour, and transforming moving time and elapsed time from seconds to minutes.

In [19]:
# Strava logs distance in meters. Let's convert this to miles.
strava_data['distance (mi)'] = strava_data['distance'] * 0.00062137
# delete the original distance column
del strava_data['distance']
# calculate miles per hour (Strava logs moving time in seconds)
strava_data['MPH'] = strava_data['distance (mi)'] / ((strava_data['moving_time']/60)/60)
# also transform moving time and elapsed time from seconds to minutes
strava_data['moving_time'] = strava_data['moving_time']/60
strava_data['elapsed_time'] = strava_data['elapsed_time']/60

I can also calculate stopping time to see how long riders took breaks.

In [20]:
strava_data['Time Stopped'] = strava_data['elapsed_time'] - strava_data['moving_time']

The number of times an athelete attempted the segement prior to a given effort might be predictive of their current moving time. Thus, I calculate the number of previous efforts by iterating through the efforts and tallying the number of the athlete's previous attempts prior to the their current effort.

I also calculate a rider's average moving time accross previous rides, as this also may help with predicting one's moving time on a later effort.

In [21]:
previous_efforts = []
previous_avgtime = []
# iterate through efforts
for ieffort in range(len(strava_data)):
    # if it's the effort, assign 0
    if ieffort == 0:
        previous_efforts.append(0)
        previous_avgtime.append(np.nan)
    else:
        this_athlete = strava_data.loc[ieffort, 'athlete']
        prev_data = strava_data.iloc[range(ieffort), :]
        # if the athlete has previous rides, count them
        # otherwise, assign 0
        if this_athlete in prev_data['athlete'].unique():
            previous_efforts.append(
                prev_data.loc[prev_data['athlete']==this_athlete,
                              'moving_time'].count()
            )
            previous_avgtime.append(
                prev_data.loc[prev_data['athlete']==this_athlete,
                              'moving_time'].mean()
            )
        else:
            previous_efforts.append(0)
            previous_avgtime.append(np.nan)

Now add metrics from previous efforts as additional features in the dataframe.

In [22]:
strava_data['Previous Efforts'] = previous_efforts
strava_data['Previous Avg Moving Time'] = previous_avgtime

Clean up some anomalous values seen in the exploratory plots. Wattage and heart rate had several 0 values (especially anomalous for heart rate!).

In [23]:
strava_data.loc[strava_data['average_watts']==0, 'average_watts'] = np.nan
strava_data.loc[strava_data['average_heartrate']==0, 'average_heartrate'] = np.nan
strava_data.loc[strava_data['max_heartrate']==0, 'max_heartrate'] = np.nan

Lastly, I'll clean up the column names a bit

In [24]:
strava_data.columns = ['ID', 'Activity', 'Athlete', 'Elapsed Time',
                       'Moving Time', 'Start Date', 'Device Watts',
                       'Avg Watts', 'Avg Heart Rate', 'Max Heart Rate', 
                       'Avg Cadence', 'Date', 'Time', 'Day', 'Hour', 
                       'Month', 'Year', 'Distance (mi)', 'MPH', 
                       'Time Stopped', 'Previous Efforts', 
                       'Previous Avg Moving Time']

Let's look at the data after cleaning up Strava features

In [25]:
strava_data.head()

,ID,Activity,Athlete,Elapsed Time,Moving Time,Start Date,Device Watts,Avg Watts,Avg Heart Rate,Max Heart Rate,...,Time,Day,Hour,Month,Year,Distance (mi),MPH,Time Stopped,Previous Efforts,Previous Avg Moving Time
0,9131432,111986,5287,76.300000,71.833333,2006-04-23 16:06:17,False,NaN,149.5,164.0,...,16:06:17,Sunday,16,4,2006,9.434820,7.880592,4.466667,0,NaN
1,9131428,125030,1885,93.783333,86.000000,2006-05-06 15:40:44,False,212.9,175.1,186.0,...,15:40:44,Saturday,15,5,2006,10.438457,7.282644,7.783333,0,NaN
2,1670259583,82155037,1885,91.566667,87.583333,2006-06-09 08:50:57,False,201.6,165.7,175.0,...,08:50:57,Friday,8,6,2006,10.513705,7.202538,3.983333,1,86.000000
3,24297526436,991534814,3719,88.666667,88.666667,2006-07-04 09:59:01,False,182.8,NaN,NaN,...,09:59:01,Tuesday,9,7,2006,9.610792,6.503543,0.000000,0,NaN
4,1670314655,82158071,1885,85.883333,83.883333,2006-07-29 08:30:41,False,226.6,172.7,180.0,...,08:30:41,Saturday,8,7,2006,10.500035,7.510456,2.000000,2,86.791667


## Clean weather history data and merge with Strava data
Load the weather history data I scraped from wunderground.com

In [26]:
weather = pd.read_csv(rawdata_fpath + 'wu_weather_history.csv')

In [27]:
weather.head()

,timestamp,Mean Temperature,Max Temperature,Min Temperature,Precipitation,Wind Speed,Max Wind Speed
0,20060101,52.0,62.0,43.0,0.02,6.0,20.0
1,20060102,52.0,54.0,48.0,0.41,5.0,14.0
2,20060103,50.0,55.0,46.0,0.00,6.0,16.0
3,20060104,55.0,60.0,50.0,0.00,6.0,12.0
4,20060105,52.0,59.0,46.0,0.00,6.0,12.0


`timestamp` is in a weird format. Let's transform it to a format that pandas' datetime method will recognize, and then transform it to a datetime object under a new column.

In [28]:
# convert timestamp to valid date representation
weather['Date'] = weather['timestamp'].apply(
    lambda x: '-'.join([str(x)[:4], str(x)[4:6], str(x)[6:]])
)
# remove timestamp column
del weather['timestamp']

# store under new 'date' column, and put it in datetime.date 
# format to match the date format in the Strava data set
weather['Date'] = pd.to_datetime(weather['Date'], 
                                 format='%Y-%m-%d', errors='coerce')
weather['Date'] = weather['Date'].apply(lambda x: x.date())

In [29]:
weather.head()

,Mean Temperature,Max Temperature,Min Temperature,Precipitation,Wind Speed,Max Wind Speed,Date
0,52.0,62.0,43.0,0.02,6.0,20.0,2006-01-01
1,52.0,54.0,48.0,0.41,5.0,14.0,2006-01-02
2,50.0,55.0,46.0,0.00,6.0,16.0,2006-01-03
3,55.0,60.0,50.0,0.00,6.0,12.0,2006-01-04
4,52.0,59.0,46.0,0.00,6.0,12.0,2006-01-05


Now, join the Strava and Weather Underground data sets, joining on 'date'

In [30]:
merged_data = pd.merge(strava_data, weather, on='Date')

In [31]:
# make sure there are no more rows than the number of values in the MPH column
merged_data = merged_data.loc[(pd.notnull(merged_data['MPH'])), :]

In [32]:
merged_data.head()

,ID,Activity,Athlete,Elapsed Time,Moving Time,Start Date,Device Watts,Avg Watts,Avg Heart Rate,Max Heart Rate,...,MPH,Time Stopped,Previous Efforts,Previous Avg Moving Time,Mean Temperature,Max Temperature,Min Temperature,Precipitation,Wind Speed,Max Wind Speed
0,9131432,111986,5287,76.300000,71.833333,2006-04-23 16:06:17,False,NaN,149.5,164.0,...,7.880592,4.466667,0,NaN,58.0,64.0,53.0,0.0,8.0,16.0
1,9131428,125030,1885,93.783333,86.000000,2006-05-06 15:40:44,False,212.9,175.1,186.0,...,7.282644,7.783333,0,NaN,58.0,68.0,51.0,0.0,15.0,17.0
2,1670259583,82155037,1885,91.566667,87.583333,2006-06-09 08:50:57,False,201.6,165.7,175.0,...,7.202538,3.983333,1,86.000000,68.0,82.0,55.0,0.0,14.0,18.0
3,24297526436,991534814,3719,88.666667,88.666667,2006-07-04 09:59:01,False,182.8,NaN,NaN,...,6.503543,0.000000,0,NaN,70.0,84.0,55.0,0.0,14.0,20.0
4,1670314655,82158071,1885,85.883333,83.883333,2006-07-29 08:30:41,False,226.6,172.7,180.0,...,7.510456,2.000000,2,86.791667,69.0,79.0,60.0,0.0,11.0,18.0


## Output cleaned data
Data look good! Now let's write the joined, cleaned data to a CSV file.

In [33]:
merged_data.to_csv(cleandata_fpath + 'strava_wu_data.csv', index=False)